# Repeated Holdout Test

**Objectives**
- Load the Model(s)
- Run predictions on the model using the Training/Validation & Holdout/Test* data split using Stratified K-Folds CV (params: k=5 & random_seed=1)
- Record the performance of the models (e.g., accuracy, precision, recall, ROC-AUC, etc.) on every CV instance for use in analysis to determine possible occurence of overfitting.

*The addition of the use of Holdout/Test split here acts as an addition to the original plan of only using Training/Validation data splits.*

**Essentially, this answers the question:** `Is the model overfitted from the training data?`

In [5]:
import pandas as pd

import lightgbm as lgbm
import catboost as catb

import sklearn.metrics as metrics 

from joblib import load

In [6]:
DF_LGBM_TB = pd.read_csv('../Dataset/TB/LGBM_TB.csv', low_memory=False) #<== Point these to the appropriate Training+Validation datasets.
DF_LGBM_IB = pd.read_csv('../Dataset/IB/LGBM_IB.csv', low_memory=False)

display(DF_LGBM_TB.head())
display(DF_LGBM_IB.head())

DF_CATB_TB = pd.read_csv('../Dataset/TB/CATB_TB.csv', low_memory=False)
DF_CATB_IB = pd.read_csv('../Dataset/IB/CATB_IB.csv', low_memory=False)

display(DF_CATB_TB.head())
display(DF_CATB_IB.head())

,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,136,135,136,135,136,135,136,135,136,...,135,141,135,141,136,135,136,135,136,135
1,1,154,177,191,154,177,191,154,134,135,...,59,138,59,138,134,135,140,134,135,134
2,1,154,177,191,154,177,191,154,134,135,...,59,138,59,138,134,135,140,134,135,134
3,0,136,135,136,135,136,135,136,135,136,...,257,193,257,193,257,193,257,193,257,193
4,1,136,135,136,135,136,135,136,135,136,...,138,59,138,59,138,209,59,138,59,138


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,136,135,183,154,177,191,134,93,153,...,148,148,148,148,148,148,148,148,148,148
1,1,154,177,191,134,135,93,153,232,56,...,148,148,148,148,148,148,148,148,148,148
2,1,154,177,191,134,135,93,153,232,56,...,148,148,148,148,148,148,148,148,148,148
3,0,136,135,183,95,262,93,153,232,236,...,148,148,148,148,148,148,148,148,148,148
4,1,136,135,183,154,177,191,134,93,153,...,148,148,148,148,148,148,148,148,148,148


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,LdrGetProcedureAddress,LookupAccountSidW,LdrGetProcedureAddress,LookupAccountSidW,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress
1,1,NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtOpenKey,NtQueryValueKey,NtClose,LdrGetDllHandle,LdrGetProcedureAddress,...,FindResourceExW,LoadResource,FindResourceExW,LoadResource,LdrGetDllHandle,LdrGetProcedureAddress,LoadStringW,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle
2,1,NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtOpenKey,NtQueryValueKey,NtClose,LdrGetDllHandle,LdrGetProcedureAddress,...,FindResourceExW,LoadResource,FindResourceExW,LoadResource,LdrGetDllHandle,LdrGetProcedureAddress,LoadStringW,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle
3,0,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,SetFilePointer,NtReadFile,SetFilePointer,NtReadFile,SetFilePointer,NtReadFile,SetFilePointer,NtReadFile,SetFilePointer,NtReadFile
4,1,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,LoadResource,FindResourceExW,LoadResource,FindResourceExW,LoadResource,OleInitialize,FindResourceExW,LoadResource,FindResourceExW,LoadResource


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,LdrLoadDll,LdrGetProcedureAddress,NtProtectVirtualMemory,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,GetSystemInfo,NtAllocateVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemInfo,NtAllocateVirtualMemory,RegOpenKeyExW,FindFirstFileExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemInfo,NtAllocateVirtualMemory,RegOpenKeyExW,FindFirstFileExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,LdrLoadDll,LdrGetProcedureAddress,NtProtectVirtualMemory,GetSystemTimeAsFileTime,SetUnhandledExceptionFilter,GetSystemInfo,NtAllocateVirtualMemory,RegOpenKeyExW,RegQueryValueExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,LdrLoadDll,LdrGetProcedureAddress,NtProtectVirtualMemory,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,GetSystemInfo,NtAllocateVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1. LightGBM

In [7]:
default_tb = load('../GBDT_Training/Outputs/Results/Demo//LGBM/Train (Default)/DEMO_LGBM_TB.model') # <== Point these to the respective .model files
default_ib = load('../GBDT_Training/Outputs/Results/Demo/LGBM/Train (Default)/DEMO_LGBM_IB.model')
tuned_tb = load('../GBDT_Training/Outputs/Results/Demo/LGBM/Train (Tuned)/TUNED_DEMO_LGBM_TB.model')
tuned_ib = load('../GBDT_Training/Outputs/Results/Demo/LGBM/Train (Tuned)/TUNED_DEMO_LGBM_IB.model')

C:\Users\ejose\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### 2. CatBoost

In [8]:
default_tb = catb.CatBoostClassifier()
default_ib = catb.CatBoostClassifier()
tuned_tb = catb.CatBoostClassifier()
tuned_ib = catb.CatBoostClassifier()
default_tb = default_tb.load_model("../GBDT_Training/Outputs/Results/Demo/CATB/Train (Default)//DEMO_CATB_TB.model", format='json') # <== Point these to the respective .model files
default_ib = default_ib.load_model("../GBDT_Training/Outputs/Results/Demo/CATB/Train (Default)/DEMO_CATB_IB.model", format='json')
tuned_tb = tuned_tb.load_model("../GBDT_Training/Outputs/Results/Demo/CATB/Train (Tuned)/TUNED_DEMO_CATB_TB.model", format='json')
tuned_ib = tuned_ib.load_model("../GBDT_Training/Outputs/Results/Demo/CATB/Train (Tuned)/TUNED_DEMO_CATB_IB.model", format='json')